### 1. SVM Hinge Loss Definition

Hinge loss is defined as follows:
<br><br>
\begin{equation}
    L_{i} = \sum_{j \neq y_{i}} = \max(0, s_{j} - s_{y_{i}} + \Delta)
\end{equation}
<br><br>
$\Delta$ is a margin value (it is an optimization hyperparameter). $s_{j}$ and $s_{y_{i}}$ are $j$th and $y_{i}$th class scores respectively for training example $x_{i}$. The class scores are computed with dot products $w_{j}^{T}x_{i}$ and $w_{y_{i}}^{T}x_{i}$.
<br><br>
The total loss to be minimized can be written as:
<br><br>
\begin{equation}
    L = \frac{1}{N}\sum_{i}^{N}L_{i} + \frac{1}{2}\lambda\|W\|_{2}^{2}
\end{equation}
<br><br>
where $\lambda$ is a regularization hyperparameter and $\frac{1}{2}$ is a constant for clean gradient computation. Intuitively, SVM wants score, $s_{y_{i}}=w_{y_{i}}^{T}x_{i}$ of the correct class $y_{i}$ to be greater than any other classes by at least the margin $\Delta$.

### 2. Computing the gradient of Hinge Loss (Unvectorized ver.)

In order to compute the gradient of the loss function w.r.t $W$, we start off with the loss for each individual training sample $x_{i}$:<br><br>
\begin{equation}
    L_{i} = \sum_{j \neq y_{i}} \max(0, w_{j}^{T}x_{i} - w_{y_{i}}^{T}x_{i} + \Delta)
\end{equation}
<br><br>
First, make sure to visualize that the wegiht matrix $W$ is of size ($D$, $C$) and our input matrix $X$ is of size ($N$, $D$) with $C$, $D$, $N$ represeting the number of classes, feature dimension, and number of train samples in $X$ respectively.
<br><br>
From the equation for $L_{i}$, we can see that we can compute the gradient of the loss by parts as follows:
<br><br>
1. When taking the gradient of loss w.r.t $w_{y_{i}}$:
<br><br>
\begin{equation}
    \nabla_{w_{y_{i}}} L_{i} = \sum_{j \neq y_{i}} \nabla_{w_{y_{i}}}\max(0, w_{j}^{T}x_{i} - w_{y_{i}}^{T}x_{i} + \Delta)
\end{equation}
<br><br>
When the hinge loss value of $w_{j}^{T}x_{i} - w_{y_{i}}^{T}x_{i} + \Delta \leq 0$, we can immediately see that $\nabla_{w_{y_{i}}} L_{i} = \nabla_{w_{y_{i}}} 0 = 0$. When it is greater than 0:
<br><br>
\begin{equation}
    \nabla_{w_{y_{i}}} L_{i} = \nabla_{w_{y_{i}}}(w_{j}^{T}x_{i} - w_{y_{i}}^{T}x_{i} + \Delta) = -x_{i}
\end{equation}
<br><br>
Now, combining the two cases, we have the following result:
<br><br>
\begin{equation}
    \nabla_{w_{y_{i}}} L_{i} = -(\sum_{j \neq y_{i}} \mathbb{1}(w_{j}^{T}x_{i} - w_{y_{i}}^{T}x_{i} + \Delta > 0))x_{i}
\end{equation}
<br><br>
where, $\mathbb{1}()$ is an indicator function for given condition.
<br><br>
2. When taking the gradient of loss w.r.t $w_{j}$:
<br><br>
\begin{equation}
    \nabla_{w_{j}} L_{i} = \sum_{j \neq y_{i}} \nabla_{w_{j}}\max(0, w_{j}^{T}x_{i} - w_{y_{i}}^{T}x_{i} + \Delta)
\end{equation}
<br><br>
With similar computations, we can see that
<br><br>
\begin{equation}
    \nabla_{w_{y_{i}}} L_{i} = \mathbb{1}(w_{j}^{T}x_{i} - w_{y_{i}}^{T}x_{i} + \Delta > 0)x_{i}
\end{equation}
<br><br>
As a final note, don't forget that hinge loss is not continuously differentiable. It is not differentiable at the point where hinge loss equals 0. 

### 3. Implementation of SVM loss and gradient computation (Naive)

In [1]:
def svm_loss_naive(W, X, y, reg):
  """
  Structured SVM loss function, naive implementation (with loops).

  Inputs have dimension D, there are C classes, and we operate on minibatches
  of N examples.

  Inputs:
  - W: A numpy array of shape (D, C) containing weights.
  - X: A numpy array of shape (N, D) containing a minibatch of data.
  - y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
  - reg: (float) regularization strength

  Returns a tuple of:
  - loss as single float
  - gradient with respect to weights W; an array of same shape as W
  """
  dW = np.zeros(W.shape) # initialize the gradient as zero

  # compute the loss and the gradient
  num_classes = W.shape[1]
  num_train = X.shape[0]
  loss = 0.0
  for i in xrange(num_train):
    # Get the class scores for ith train sample.
    scores = X[i].dot(W)
    # Set aside the correct class score for margin computation.
    correct_class_score = scores[y[i]]
    for j in xrange(num_classes):
      if j == y[i]:
        # Since hinge loss does not look at the correct class, we skip.
        continue
      margin = scores[j] - correct_class_score + 1 # note delta = 1
      if margin > 0:
        loss += margin
        # Note W.shape = (D, C)
        # Note X.shape = (N, D)
        dW[:, y[i]] -= X[i, :]
        dW[:, j] += X[i, :]
  # Normalize the total loss with respect to num_train.
  loss /= num_train
  # Normalize the gradient with respect to num_train
  dW /= num_train
  # Add the regularization term to the total loss.
  loss += 0.5 * reg * np.sum(W * W)
  # Add the gradient of the regularization term to the gradient.
  dW += reg * W
    
  return loss, dW

### 4. Vectorized SVM hinge loss and gradient implementation

The gradient derivation presented above is still applied in the vectorized implementation. Rather than computing the class score for each sample, we can compute the class scores for all training samples in $X$ by taking the dot product with weights $W$. Let's visualize the two matrices:
<br><br>
$$
X =
\begin{bmatrix}
    x_{11} & x_{12} & x_{13} & \dots  & x_{1D} \\
    x_{21} & x_{22} & x_{23} & \dots  & x_{2D} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    x_{N1} & x_{N2} & x_{N3} & \dots  & x_{ND}
\end{bmatrix}, 
W =
\begin{bmatrix}
    w_{11} & w_{21} & w_{31} & \dots  & w_{C1} \\
    w_{12} & w_{22} & w_{32} & \dots  & w_{C2} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    w_{1D} & w_{2D} & w_{3D} & \dots  & w_{CD}
\end{bmatrix}
$$
<br><br>
Recall that each row of $X$ represents a training sample and each column of $W$ represents a weight vector for a class. Therefore, the dot product of the two:
<br><br>
$$
XW =
\begin{bmatrix}
    s_{11} & s_{12} & s_{13} & \dots  & s_{1C} \\
    s_{21} & s_{22} & s_{23} & \dots  & s_{2C} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    s_{N1} & s_{N2} & s_{N3} & \dots  & s_{NC}
\end{bmatrix}
$$


In [1]:
def svm_loss_vectorized(W, X, y, reg):
  """
  Structured SVM loss function, vectorized implementation.

  Inputs and outputs are the same as svm_loss_naive.
  """
  loss = 0.0
  num_train = X.shape[0]
  dW = np.zeros(W.shape) # initialize the gradient as zero
  scores = X.dot(W)
  correct_scores = scores[np.arange(scores.shape[0]), y]
  margin = np.maximum(0, scores - np.matrix(correct_scores).T + 1)
  margin[np.arange(num_train), y] = 0
  loss = np.mean(np.sum(margin, axis=1))
  loss += 0.5 * reg * np.sum(W * W)
  
  # As we have derived from the previous section,
  # We need an indicator matrix for margins that are greater than 0.
  indicators = margin
  indicators[margin > 0] = 1
  # For the gradient of the loss w.r.t correct class weight,
  # we must sum 1's across the columns of each row.
  row_sum = np.sum(indicators, axis=1)
  indicators[np.arange(num_train), y] = -row_sum.T
  dW = X.T.dot(indicators)
  dW /= num_train
  dW += reg * W

  return loss, dW